In [77]:
require('tidyverse')
options(stringsAsFactors=FALSE)

In [78]:
# need to sample a bunch of null variants to get this set up

In [79]:
trait <- "arm_fp"
m_v <- read_tsv(sprintf("../gwas_valid/ukb24983_v2_hg19.%s_onesex.genotyped.glm.linear", trait) ) 
f_v <- read_tsv(sprintf("../gwas_valid/ukb24983_v2_hg19.%s_zerosex.genotyped.glm.linear", trait))

df_t_train <- read.table(sprintf("../data/1020/m2/snps4_%s.txt", trait), sep=" ", header=TRUE); 
m_train <- df_t_train %>% filter(category==3) %>% select( -p1, -p2, -p4, -category) %>% rename(post=p3, ID=SNP)
m_train <- filter(m_train, post > 0.8)
f_train <- df_t_train %>% filter(category==2) %>% select( -p1, -p3, -p4, -category)%>% rename(post=p2, ID=SNP)
f_train <- filter(f_train, post > 0.8)
nrow(f_train)
nrow(m_train)
load(sprintf("../data/1020/dat_%s.RData", trait))

Parsed with column specification:
cols(
  `#CHROM` = col_character(),
  POS = col_integer(),
  ID = col_character(),
  REF = col_character(),
  ALT = col_character(),
  A1 = col_character(),
  TEST = col_character(),
  OBS_CT = col_integer(),
  BETA = col_double(),
  SE = col_double(),
  T_STAT = col_double(),
  P = col_double()
)
Parsed with column specification:
cols(
  `#CHROM` = col_character(),
  POS = col_integer(),
  ID = col_character(),
  REF = col_character(),
  ALT = col_character(),
  A1 = col_character(),
  TEST = col_character(),
  OBS_CT = col_integer(),
  BETA = col_double(),
  SE = col_double(),
  T_STAT = col_double(),
  P = col_double()
)


[1] 528

[1] 13

In [80]:
head(f_train)

ID,CHR,B.f,B.m,SE.f,SE.m,P.f,P.m,post,gene
Affx-13928922,17,-0.0167939,-0.01042940,1.747633e-05,1.876458e-05,5.89147e-05,0.01605760,0.8645675,MAPT
Affx-22413417,3,0.0184675,0.01056600,1.475244e-05,1.549580e-05,1.52485e-06,0.00727276,0.9946475,LOC100129060
Affx-37516233,9,-0.0421555,0.00973972,1.032866e-04,1.076655e-04,3.35637e-05,0.34790900,0.9244661,AIF1L
rs1006217,10,0.0191111,-0.00894054,2.071624e-05,2.161794e-05,2.68399e-05,0.05449430,0.9333225,DOCK1
rs10064229,5,0.0261723,0.00390157,4.203824e-05,4.448370e-05,5.42446e-05,0.55856400,0.8956191,LMAN2
rs10077222,5,-0.0355189,-0.00111383,7.187021e-05,7.603544e-05,2.79436e-05,0.89835800,0.9398192,NA


In [81]:
# sample a bunch of null variants

NUM.NULL <- 10000
null_vars <- setdiff(dat$snp, df_t_train$SNP)


In [82]:
head(null_vars)

[1] "rs6649864"   "rs56328755"  "rs28669107"  "rs148672604" "rs146939120"
[6] "rs28510852"

In [83]:
null_idx <- sample(1:length(null_vars), NUM.NULL, replace=FALSE)
sampled_null_vars <- null_vars[null_idx]

In [84]:
head(sampled_null_vars)

[1] "rs62363581"  "rs9536283"   "rs115242197" "rs75297243"  "rs16920119" 
[6] "rs7222500"

In [85]:
# get the non-null data from both disc and valid
sel_rows <- which(dat$snp %in% sampled_null_vars)
null_B_disc <- dat$dat$B[sel_rows,]
null_SE_disc <- dat$dat$SE[sel_rows,]
snp_ids <- dat$snp[sel_rows]
pvals <- dat$p[sel_rows]

In [86]:

rownames(m_v) <- m_v$ID
rownames(f_v) <- f_v$ID
null_m_v <- m_v[snp_ids,]
null_f_v <- f_v[snp_ids,]

null_B_val <- cbind(null_f_v$BETA, null_m_v$BETA)
null_SE_val <- cbind(null_f_v$SE, null_m_v$SE)

Warning message:
“Setting row names on a tibble is deprecated.”Warning message:
“Setting row names on a tibble is deprecated.”

In [87]:
setupGenCorr <- function(B, SE, run_name){
    dat <- list("K"=2, "M"=2, "N"=nrow(B), "B"=B, "SE"=SE)
    save(dat, file=sprintf("../data/valid_calc/dat_%s.RData", run_name))
}


In [88]:
  m_train <- df_t_train %>% filter(category==3) %>% select( -p1, -p2, -p4, -category) %>% rename(post=p3, ID=SNP)
    f_train <- df_t_train %>% filter(category==2) %>% select( -p1, -p3, -p4, -category)%>% rename(post=p2, ID=SNP)
    
    test_valid <- cbind( (f_v %>% select(`#CHROM`, BETA, SE, P) %>% rename(CHR=`#CHROM`, B.f=BETA, SE.f=SE, P.f=P)),
        (m_v %>% select(ID, POS, REF, ALT, BETA, SE, P) %>% rename(B.m=BETA, SE.m=SE, P.m=P)))

    test_valid_filt <- test_valid %>% filter(ID %in% c(f_train$ID, m_train$ID)) %>% 
    mutate(sex=ifelse(ID %in% f_train$ID, "female", "male"), dataset="validation")

    test_valid_filt2 <- test_valid_filt %>% select(-POS, -REF, -ALT)
    test_disc <- rbind(f_train %>% mutate(sex="female"), m_train %>% mutate(sex="male")) %>% mutate(dataset="discovery")


    test_valid_filt3 <- test_valid_filt2 %>% 
    rename(B.f.v=B.f, B.m.v=B.m, SE.f.v=SE.f, SE.m.v=SE.m, P.f.v=P.f, P.m.v=P.m) %>% 
    select(-dataset) 

    train_valid_compare <- full_join(test_disc %>% select(-sex, -dataset), test_valid_filt3 %>% select( -CHR), by="ID")
compare_df <- train_valid_compare

In [89]:
head(compare_df)

ID,CHR,B.f,B.m,SE.f,SE.m,P.f,P.m,post,gene,B.f.v,SE.f.v,P.f.v,B.m.v,SE.m.v,P.m.v,sex
Affx-11234541,14,0.0249440,0.000421050,4.766632e-05,4.949488e-05,3.02836e-04,0.95227600,0.6510781,EVL,0.0183904,0.0241863,0.4470500,-0.01591600,0.0265409,0.548734,female
Affx-13919467,17,0.0201922,0.009009520,2.901234e-05,3.051756e-05,1.77737e-04,0.10291400,0.7297835,NA,0.0335390,0.0194113,0.0840438,0.00729871,0.0209423,0.727461,female
Affx-13928922,17,-0.0167939,-0.010429400,1.747633e-05,1.876458e-05,5.89147e-05,0.01605760,0.8645675,MAPT,-0.0281355,0.0142421,0.0482309,-0.02509650,0.0159598,0.115872,female
Affx-22413417,3,0.0184675,0.010566000,1.475244e-05,1.549580e-05,1.52485e-06,0.00727276,0.9946475,LOC100129060,0.0101006,0.0140044,0.4707730,-0.00855645,0.0150366,0.569340,female
Affx-25624255,5,-0.0312970,-0.007689610,6.990231e-05,7.289215e-05,1.81663e-04,0.36776800,0.7517303,LYRM7,-0.0363408,0.0294367,0.2170230,0.00109717,0.0330212,0.973495,female
Affx-2643209,10,0.0187879,0.000969214,2.559055e-05,2.692489e-05,2.04097e-04,0.85182900,0.6966080,NA,0.0107016,0.0176394,0.5440700,0.00337732,0.0195987,0.863187,female


In [90]:
f.vars <- compare_df %>% filter(sex=="female")

# ---- b_f, b_f_v ---- #
      B_FF <- cbind(f.vars$B.f, f.vars$B.f.v)
      SE_FF <- cbind(f.vars$SE.f, f.vars$SE.f.v)
       b_null_f <-  cbind(null_B_disc[,1], null_B_val[,1])
       se_null_f <-  cbind(null_SE_disc[,1], null_SE_val[,1])
head(B_FF)
head(b_null_f)
nrow(rbind(B_FF, b_null_f))
table(apply(B_FF, 1, function(x) any(is.na(x))))


0.0249440,0.0183904
0.0201922,0.0335390
-0.0167939,-0.0281355
0.0184675,0.0101006
-0.0312970,-0.0363408
0.0187879,0.0107016


0.002257220,-0.029506000
-0.003376700,-0.004855050
0.006383090,-0.011569800
-0.005394510,0.007944030
0.003513950,0.000516363
0.000842712,0.021157300


[1] 11168


FALSE 
 1168 

In [91]:


na.rows <- which(apply(b_null_f, 1, function(x) any(is.na(x))))
head(na.rows)
    
b_null_f <- b_null_f[-na.rows,]
se_null_f <- se_null_f[-na.rows,]
    


[1] 1692 2215 2411 3220 3454 6047

In [92]:
       b_null_m <-  cbind(null_B_disc[,2], null_B_val[,2])
       se_null_m <-  cbind(null_SE_disc[,2], null_SE_val[,2])
b_null_m <- b_null_m[-na.rows,]
se_null_m <- se_null_m[-na.rows,]

na.rows.m <- which(apply(b_null_m, 1, function(x) any(is.na(x))))

    
b_null_m <- b_null_m[-na.rows.m,]
se_null_m <- se_null_m[-na.rows.m,]
b_null_f <- b_null_f[-na.rows.m,]
se_null_f <- se_null_f[-na.rows.m,]
    


In [93]:
    
######## F-SPECIFIC #########    
f.vars <- compare_df %>% filter(sex=="female")

# ---- b_f, b_f_v ---- #
      B_FF <- cbind(f.vars$B.f, f.vars$B.f.v)
      SE_FF <- cbind(f.vars$SE.f, f.vars$SE.f.v)

    g1 <- setupGenCorr(rbind(B_FF, b_null_f), rbind(SE_FF, se_null_f), sprintf("%s_fspec_ff", trait))
     

# ---- b_m, b_m_v ---- #
      B_FM <- cbind(f.vars$B.m, f.vars$B.m.v)
      SE_FM <- cbind(f.vars$SE.m, f.vars$SE.m.v) 

 g2 <- setupGenCorr(rbind(B_FM, b_null_m), rbind(SE_FM, se_null_m), sprintf("%s_fspec_mm", trait))


######## M-SPECIFIC #########    
m.vars <- compare_df %>% filter(sex=="male")

# ---- b_f, b_f_v ---- #
      B_MF <- cbind(m.vars$B.f, m.vars$B.f.v)
      SE_MF <- cbind(m.vars$SE.f, m.vars$SE.f.v)
    g3 <- setupGenCorr(rbind(B_MF, b_null_f), rbind(SE_MF, se_null_f), sprintf("%s_mspec_ff", trait))

# ---- b_m, b_m_v ---- #
      B_MM <- cbind(m.vars$B.m, m.vars$B.m.v)
      SE_MM <- cbind(m.vars$SE.m, m.vars$SE.m.v) 

 g4 <- setupGenCorr(rbind(B_MM, b_null_m), rbind(SE_MM, se_null_m), sprintf("%s_mspec_mm", trait))
